In [49]:
import random

Alice_input = 1  # Alice's input bit
Bob_input = 0    # Bob's input bit

# Making the Shares
def share(secret: int) -> tuple[int, int]:
    # Secret share a bit 'secret' between Alice and Bob.
    x_b = random.randint(0, 1)  # Bob's share
    x_a = secret ^ x_b           # Alice's share
    return (x_a, x_b)

# Reconstructing the secret from shares
def reconstruct(share: tuple[int, int]) -> int:
    # Reconstruct the bit from a sharing [x] = (xA, xB).
    x_a, x_b = share
    return x_a ^ x_b

alice_share = share(Alice_input)
bob_share = share(Bob_input)
print("Alice input:")
print(Alice_input)
print("Bob input:")
print(Bob_input)

Alice input:
1
Bob input:
0


In [ ]:
# Individual Gates
def xor_const(share: tuple[int, int], const: int) -> tuple[int, int]:
    # const = 1 is the NOT gate
    x_a, x_b = share
    z_a = x_a ^ const
    z_b = x_b
    return (z_a, z_b)

print(reconstruct(xor_const(alice_share, 0)))
print(reconstruct(xor_const(bob_share, 0)))

def and_const(share: tuple[int, int], const: int) -> tuple[int, int]:
    # const = 0, zeros the gate
    x_a, x_b = share
    z_a = const * x_a
    z_b = const * x_b
    return (z_a, z_b)

print(reconstruct(and_const(alice_share, 1)))
print(reconstruct(and_const(bob_share, 1)))

# Two-Wire Gates
def xor_gate(share1: tuple[int, int], share2: tuple[int, int]) -> tuple[int, int]:
    x_a, x_b = share1
    y_a, y_b = share2
    z_a = x_a ^ y_a
    z_b = x_b ^ y_b
    return (z_a, z_b)

print(reconstruct(xor_gate(alice_share, bob_share)))  # Expected output: 0


1
0
1
0
1


In [50]:
# Dealer Gates
def dealer() -> tuple[tuple[int, int], tuple[int, int], tuple[int, int]]:
    # Dealer provides random bits u, v, w such that w = u * v
    u = random.randint(0, 1)
    v = random.randint(0, 1)
    w = u * v
    return share(u), share(v), share(w)

def and_gate(share1: tuple[int, int], share2: tuple[int, int]) -> int:
    # u, v, w are from the dealer
    (x_a, x_b), (y_a, y_b) = share1, share2
    (u_a, u_b), (v_a, v_b), (w_a, w_b) = dealer()

    d = xor_gate((x_a, x_b), (u_a, u_b))
    e = xor_gate((y_a, y_b), (v_a, v_b))

    # d and e become public
    d = reconstruct(d)
    e = reconstruct(e)

    z_a = w_a ^ (e * x_a) ^ (d * y_a) ^ (d * e)
    z_b = w_b ^ (e * x_b) ^ (d * y_b)
    return (z_a, z_b)

print(reconstruct(and_gate(alice_share, bob_share)))

0
